#  <center> Dokumentacja projektu Sentiment Classification
<center> <b> Aleksander Ogonowski
   
<center> Paweł Rybak

<center> Kornel Szymczyk 267778 </b>

<br>
<center> <i> 10 stycznia 2019 </i>


## 1. Instrukcja użytkownika

Pakiet pszt zawiera dwa moduły:
- refactor_csv.py zawierający funkcje pomocnicze do refaktoryzacji zbioru danych Tweetów dotyczących produktów Apple,
- net.py zawierający klasę MLP, która jest definicją modelu sieci neuronowej.

### 1.1. Wymagania uruchomienia
- Python 3.6+
- Numpy

### 1.2. Przekształcenie zbioru danych wykorzystując moduł refactor_csv.py

Przed przystąpieniem do trenowania sieci neuronowej należy przekrzstałcić odpowiednio zbiór danych i zapisać go w nowym pliku csv. W tym celu korzystamy z funkcji *refactor* podając jako pierwszy argument ściężkę do pliku ze zbiorem danych csv oraz jako drugi argument ścieżke pliku, w którym zostanie zapisany przekształcony zbiór. 

Przykład kodu:

In [1]:
from pszt import refactor_csv

# File path to the csv file.
csv_path = 'Apple-Twitter-Sentiment-DFE.csv'

# File path to the new csv file.
new_csv_path = 'apple-twitter_example.csv'
    
refactor_csv.refactor(csv_path, new_csv_path)

Posiadając przekształcony zbiór plik csv należy przystąpić do wektoryzacji zbioru, w tym celu należy skorzystać z funkcji *refactor_csv*. Jej celem jest wykonanie przekształcenia zdań do modelu bag-of-words oraz zamiana sentymentu zdań (1, 3, 5) do postaci one-hot. Przekształcone zdania zapisywane są w tablicy numpy *x_train*, etykiety w tablicy numpy *y_train* oraz zapisywane są na dysku w formacie *.npy*. Jako pierwszy argument funkcja przyjmuję ścieżkę do pliku csv utworzonego przy pomocy funkcji *refactor*, jako drugi argument ścieżkę pliku do zapisania tablicy *x_train*, jako trzeci argument ścieżkę pliku do zapisania tablicy *y_train* oraz ostatni argument przyjmuje opcjonalną tablicę słów, które mają być pomijane w zdaniach.

Przykład kodu:

In [ ]:
refactor_csv.vectorize_dataset(new_csv_path, 'x_train_example', 'y_train_example', ['a', 'apple'])

### 1.3. Trenowanie sieci neuronowej wykorzystując moduł net.py

W celu wytrenowania sieci tworzymy obiekt klasy MLP. Następnie dodajemy warstwy sieci wykorzystując metodę *add_layer(input_dim, output_dim, activation)*. Pierwszy argument określa liczbę wejść do warstwy, drugi argument określa liczbę wyjść z warstwy, a ostatni określa rodzaj funkcji aktywacji ('relu', 'sigmoid', 'softmax'). Ze względu na to, że etykiety zbioru reprezentowane są w postaci one-hot, ostatnia warstwa powinna zawsze posiadać output_dim=3 oraz activation='softmax'. Model sieci został napisany w taki sposób, że należy pominąć dodawanie warstwy wejściowej sieci neurnowej, a input_dim pierwszej warstwy powinien odpowiadać długości wektora danych wejściowych. Po dodaniu warstw należy je zainicjalizować funkcją *init_layers()*.

Proces trenowania wywołuje się funkcją *train(x, y_true, epochs, silent=False)*. Pierwszy argument to tablica numpy ze zdaniami w postaci bag-of-words, drugi argument to tablicy etykiet zdań w postaci one-hot, trzeci argument określa liczbę kroków trenowania, czwarty agrument opcjonalny określający czy wypisywać na ekran aktualny status trenowania sieci.

Przykład kodu:

In [ ]:
import numpy as np
from pszt import net

# Create a neural net.
mlp = net.MLP();

# Load dataset.
x_train = np.load('x_train_3k.npy')
y_train = np.load('y_train_3k.npy')

# Add layers.
mlp.add_layer(input_dim=x_train.shape[1], output_dim=20, activation="sigmoid")
mlp.add_layer(input_dim=20, output_dim=3, activation="softmax")

# Initialize weights in layers.
mlp.init_layers()

# Train the neural net.
mlp.train(x=x_train, y_true=y_train, epochs=5)

Klasa MLP zawiera również metodę *k_fold_validation(x, y_true, k, epochs)*, która poddaje sieć neuronową walidacji k-fold.

Przykład kodu:

In [ ]:
mlp.k_fold_validation(x=x_train, y_true=y_train, k=5, epochs=5)

## 2. Struktura programu

# TODO: Opisać jak działa sieć. Najlepiej głównie skupić się na init_weights, forward i backward.

## 3. Kluczowe decyzje projektowe.



## 4. Wnioski dotyczące osiągniętych rezultatów